In [4]:
# https://api.crunchbase.com/v3.1/organizations?user_key=662e263576fe3e4ea5991edfbcfb9883
# To do this, you just add "news" as a relationship query parameter for any specific organization or person
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import nltk
from nltk import *
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
import codecs
import string
import random
import re

import requests
import urllib
import bs4
from bs4 import BeautifulSoup
from readability.readability import Document # https://github.com/buriy/python-readability. Tried Goose, Newspaper (python libraries on Github). Bad results.
from http.cookiejar import CookieJar # 

import pycrunchbase
cb = pycrunchbase.CrunchBase('662e263576fe3e4ea5991edfbcfb9883')

In [9]:
rootdir = '/Users/ksun/downloads/startup-data/ai-organizations/'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if '.DS_Store' not in file:
            print(subdir.split("/")[-1] + " | " + file)

2021-ai | https---www-nanalyze-com-2019-01-artificial-intelligence-denmark-.txt
2021-ai | https---2021-ai-2021-ai-launches-ai-platform-apex-.txt
2021-ai | https---2021-ai-strategic-partnership-qvartz-.txt
2021-ai | https---www-cio-com-au-mediareleases-31691-2021ai-launches-ai-platform-apex-to-empower-.txt
2021-ai | https---2021-ai-partnering-google-cloud-deliver-ai-technology-enterprises-.txt
alchemy-iot | http---www-builtincolorado-com-2017-02-22-colorado-iot-startups.txt
etsimo-ltd | https---dl-acm-org-citation-cfm?id=3210172.txt
etsimo-ltd | https---www-dr-hempel-network-com-digital_health_contact_lists-122-digital-health-innovators-ceos-in-finland-.txt
tinc-works | https---www-linkedin-com-feed-update-urn-li-activity-6426044812574625792.txt
tinc-works | https---www-linkedin-com-feed-update-urn-li-activity-6447466227475582978.txt
tinc-works | https---cloudworks-nu-2018-09-26-tinc-works-versterkt-team-om-de-groei-te-versnellen-.txt
tinc-works | https---cloudworks-nu-2019-01-21-met-de

gecko-robotics | http---blog-ycombinator-com-hard-tech-startups.txt
gecko-robotics | http---techcrunch-com-2016-03-23-gecko-robotics-does-the-heavy-inspecting-at-power-plants-to-prevent-human-fatalities-.txt
gecko-robotics | http---blog-ycombinator-com-gecko-robotics-yc-w16-builds-robots-to-automate-infrastructure-inspections.txt
gecko-robotics | http---www-chron-com-business-article-Texas-A-M-students-win-Rice-Business-Plan-7257972-php.txt
gecko-robotics | http---www-businessinsider-com-big-ideas-during-199-pitches-at-y-combinator-demo-day-2016-3.txt
rekener | https---www-rekener-com-blog-sales-analytics-HubSpot-CRM.txt
rekener | https---www-rekener-com-blog-stage-duration-in-sales-process.txt
rekener | https---www-rekener-com-blog-sales-coverage-analytics-account-based-selling.txt
rekener | https---www-rekener-com-blog-account-based-sales-B2B-inside-sales.txt
rekener | https---www-rekener-com-blog-stage-conversion-rate-analysis.txt
rekener | https---www-rekener-com-blog-B2B-sales-dat

youper | http---super-abril-com-br-saude-aplicativo-brasileiro-promete-ajudar-no-tratamento-da-ansiedade-.txt
youper | https---techcrunch-com-2018-10-09-brazils-healthtech-sector-is-new-hot-spot-.txt
youper | https---www-equidam-com-top-10-brazilian-startups-november-2016-.txt
saleswings | http---startupticker-ch-en-news-april-2016-epfl-alumni-seed-night-2016.txt
favizone | https---www-frenchweb-fr-fw-radar-favizone-lassistant-shopping-pour-les-e-commercants-302217.txt
asaii | https---www-billboard-com-articles-business-8479863-apple-acquires-asaii-data-analytics-music.txt
asaii | https---www-fool-com-investing-2018-10-21-is-apple-building-an-ai-powered-music-label-aspx.txt
asaii | https---www-theverge-com-2018-10-15-17977616-apple-acquires-asaii-machine-learning-music-analytics-spotify.txt
asaii | https---t-co-lMB2RG4Be4?amp=1.txt
asaii | https---www-edmtunes-com-2018-10-apple-growing-on-big-data-acquiring-company-to-find-new-talents-.txt
asaii | https---www-fool-com-investing-2018-10

banuba | http---www-ewdn-com-2018-12-03-russian-and-belarusian-investors-back-ar-technology-developer-banuba-?share=telegram.txt
banuba | https---techcrunch-com-2018-11-26-banuba-raises-7m-to-supercharge-any-app-or-device-with-the-ability-to-really-see-you-.txt
banuba | https---www-nanalyze-com-2018-08-top-10-artificial-intelligence-startups-hong-kong-.txt
cerevrum-inc | https---virtualrealityreporter-com-cerevrum-spaceship-3d-vr-cognitive-learning-environment-.txt
cerevrum-inc | http---www-roadtovr-com-new-gear-vr-games-and-apps-january-week-2-.txt
cerevrum-inc | http---touchstoneresearch-com-this-week-in-cool-new-samsung-gear-vr-game-reviews-polyrunner-vr-cerevrum-.txt
cerevrum-inc | http---uploadvr-com-gear-vr-cerevrum-star-chart-.txt
cerevrum-inc | https---www-vrfocus-com-2016-01-vrarlab-launches-the-first-vr-brain-training-title-cerevrum-on-gear-vr-.txt
cerevrum-inc | http---vrscout-com-news-motion-controllers-best-solution-vr-today-.txt
sykopomp-industries | https---www-google-co

In [ ]:
# url = 'https://api.crunchbase.com/v3.1/csv_export/csv_export.tar.gz?user_key=662e263576fe3e4ea5991edfbcfb9883'

In [ ]:
url = 'https://api.crunchbase.com/v3.1/organizations/?founded_on=2018-01-01&user_key=662e263576fe3e4ea5991edfbcfb9883'
# location given here 
location = "philadephia"
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
response = requests.get(url, PARAMS)
response.json()
    

Just to check if all works well, try the following quick exercise. Access the CB API (with my login info), which is inside Dropbox. Download the list of companies founded in 2010-2014 and with category ‘Artificial Intelligence’. I suspect there will be something in the order of 2000sh companies like that.
 
I will email the person the HR person this week and cc you, so you can coordinate an appointment. I am just waiting a couple more days, so that perhaps Tiffany will join in too and we can do one meeting only.
 
As soon as we’re done with the paperwork, we’ll get right into the project. Looking forward to.

In [ ]:
url = 'https://api.crunchbase.com/v3.1/organizations?user_key=662e263576fe3e4ea5991edfbcfb9883'
# location given here 
location = "philadephia"
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
response = requests.get(url, PARAMS)
response.json()

In [ ]:
df = json_normalize(response.json())
df

In [ ]:
organizations = cb.organizations("");
[org for org in organizations]

In [ ]:
github = cb.organization('github')

In [ ]:
github

In [ ]:
github_team = github.current_team
in_the_news = github.news
print(github_team)
who_started_github = github.founders
print(who_started_github)
all_founders = [founder for founder in who_started_github]
print(all_founders)
print(in_the_news)

In [ ]:
all_news_urls = []
more_news = github.news
while(True):
    more_news = cb.more(more_news)
    news_urls = [news.url for news in more_news]
    all_news_urls = all_news_urls + news_urls[0:5]
    print(more_news)
    print(len(all_news_urls))
    break

In [81]:
df = pd.read_csv('csv_export/new_organizations2.csv')
df = pd.DataFrame(df, columns=['company_name', 'permalink',
       'short_description', 'category_list'])
df.shape

(1158, 4)

In [82]:
# only tech companies, maybe 5 articles for each?
df['news_article_links'] = ''
df['article'] = ''
df['sentiment_polarity'] = 0 # the sentiment polarity within an article (range of emotions)

In [83]:
df

,company_name,permalink,short_description,category_list,news_article_links,article,sentiment_polarity
0,Intel,/organization/intel,"Intel designs, manufactures, and sells integra...","Artificial Intelligence,Consumer Electronics,E...",,,0
1,Oracle,/organization/oracle,Oracle is an integrated cloud applications and...,"Collaboration,Computer,Enterprise Software,Inf...",,,0
2,Kainos,/organization/kainos-software,Kainos provide digital technology solutions th...,"Consulting,Financial Services,FinTech,Health C...",,,0
3,Progress,/organization/progress-software,Progress is a global software company that sim...,"Computer,Enterprise Software,Information Techn...",,,0
4,Infinium Software,/organization/software-2000,Infinium Software provides enterprise business...,"Enterprise Software,Information Services,Infor...",,,0
5,NetApp,/organization/netapp,NetApp is a hybrid cloud data services company .,"Cloud Computing,Data Storage,Information Techn...",,,0
6,Veon,/organization/vimpelcom,Veon is an international communications and te...,"Information Technology,Internet,Telecommunicat...",,,0
7,Yahoo Pro,/organization/yahoo-pro,Yahoo is a technology company that is known fo...,"Advertising,Consumer,Email,Finance,Financial S...",,,0
8,Euronet Worldwide,/organization/euronet-worldwide,Euronet Worldwide provides electronic payment ...,"E-Commerce,Email,Finance,Financial Services,In...",,,0
9,Clarus Corporation,/organization/clarus-corporation-2,A comprehensive suite of world-class financial...,"Enterprise Software,Human Resources,Informatio...",,,0


In [106]:
# companyName = 'powerdsine'
# company = cb.organization(companyName)
# all_news_urls = []
# more_news = company.news

# more_news = cb.more(more_news)
# news_urls = [news.url for news in more_news]
# print(news_urls)
df = pd.read_csv('csv_export/temp1.5.csv')
df = pd.DataFrame(df, columns=['company_name', 'permalink',
       'short_description', 'category_list', 'news_article_links', 'article', 'sentiment_polarity'])
df.head()

,company_name,permalink,short_description,category_list,news_article_links,article,sentiment_polarity
0,Intel,/organization/intel,"Intel designs, manufactures, and sells integra...","Artificial Intelligence,Consumer Electronics,E...",http://www.cbj.ca/development-timelines-and-tr...,"\nCBJ Newsmakers\nVANCOUVER, British Columbia,...",0
1,Oracle,/organization/oracle,Oracle is an integrated cloud applications and...,"Collaboration,Computer,Enterprise Software,Inf...",https://techcrunch.com/2018/11/29/the-white-ho...,\n\nThe White House will meet with tech execs ...,0
2,Kainos,/organization/kainos-software,Kainos provide digital technology solutions th...,"Consulting,Financial Services,FinTech,Health C...",https://www.pehub.com/2019/01/pe-hub-wire-high...,\n\n \n\n\t\t\t\t\t\t\t\tPhoto of Luisa Beltra...,0
3,Progress,/organization/progress-software,Progress is a global software company that sim...,"Computer,Enterprise Software,Information Techn...",https://www.huffingtonpost.com/entry/max-rose-...,"\nDemocrat Max Rose, a 31-year-old former Army...",0
4,Infinium Software,/organization/software-2000,Infinium Software provides enterprise business...,"Enterprise Software,Information Services,Infor...",NaN,NaN,0


In [128]:
pd.read_csv('csv_export/temp1.5.csv')

,company_name,roles,permalink,country_code,state_code,region,city,address,status,short_description,...,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type,news_article_links,article,sentiment_polarity
0,Intel,"company,investor",/organization/intel,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,ipo,"Intel designs, manufactures, and sells integra...",...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2/6/08 6:28,12/11/18 10:52,company,organization,http://www.cbj.ca/development-timelines-and-tr...,"\nCBJ Newsmakers\nVANCOUVER, British Columbia,...",0
1,Oracle,"company,investor",/organization/oracle,USA,CA,SF Bay Area,Redwood Shores,500 Oracle Parkway,ipo,Oracle is an integrated cloud applications and...,...,https://twitter.com/Oracle,NaN,bef5bd4b-72c6-7877-d7ab-8bbe43f7bda7,2/28/08 0:16,1/4/19 9:07,company,organization,https://techcrunch.com/2018/11/29/the-white-ho...,\n\nThe White House will meet with tech execs ...,0
2,Kainos,"company,investor",/organization/kainos-software,GBR,NaN,Belfast,Belfast,4-6 Upper Crescent,ipo,Kainos provide digital technology solutions th...,...,http://twitter.com/KainosSoftware,Kainos Group Plc,a307ed6d-2605-328a-e50a-d83f82871ac7,8/15/13 14:40,4/17/18 9:29,company,organization,https://www.pehub.com/2019/01/pe-hub-wire-high...,\n\n \n\n\t\t\t\t\t\t\t\tPhoto of Luisa Beltra...,0
3,Progress,company,/organization/progress-software,USA,MA,Boston,Bedford,14 Oak Park Drive,ipo,Progress is a global software company that sim...,...,https://twitter.com/ProgressSW,Progress Software,ac1b64f5-4012-c3f9-59c0-1f48f1df3f5c,4/10/08 7:39,10/2/18 23:54,company,organization,https://www.huffingtonpost.com/entry/max-rose-...,"\nDemocrat Max Rose, a 31-year-old former Army...",0
4,Infinium Software,company,/organization/software-2000,USA,MA,NaN,Hyannis Port,NaN,ipo,Infinium Software provides enterprise business...,...,NaN,Software 2000,01915de2-7ae6-9787-e397-855a71b1fafe,9/13/13 12:45,2/12/18 23:35,company,organization,NaN,NaN,0
5,NetApp,"company,investor",/organization/netapp,USA,CA,SF Bay Area,Sunnyvale,495 East Java Drive,ipo,NetApp is a hybrid cloud data services company .,...,http://twitter.com/netapp,Net App,bbd3ae58-58e2-d81c-42bf-97d4a07ac0d1,1/4/08 15:41,11/27/18 21:57,company,organization,https://www.prnewswire.com/news-releases/liqui...,"\nLANSING, Mich., Jan. 29, 2019 /PRNewswire/ -...",0
6,Veon,company,/organization/vimpelcom,NLD,NaN,Amsterdam,Amsterdam,"SOM 2 Bld., Floor 2, Claude Debussylaan 15",ipo,Veon is an international communications and te...,...,https://twitter.com/veondigital,"FGI Wireless,VimpelCom",3565d953-9352-8281-302d-721fd05a0f6a,5/6/11 19:15,10/27/18 0:25,company,organization,https://www.prnewswire.com/news-releases/veon-...,"\nAMSTERDAM, Jan. 23, 2019 /PRNewswire/ -- VEO...",0
7,Yahoo Pro,company,/organization/yahoo-pro,USA,CA,NaN,Sunnyvale,NaN,ipo,Yahoo is a technology company that is known fo...,...,https://twitter.com/Yahoo,NaN,c30b2e7d-a46e-4c22-9533-c27efff502ae,11/12/18 21:16,11/13/18 18:42,company,organization,NaN,NaN,0
8,Euronet Worldwide,"company,investor",/organization/euronet-worldwide,USA,KS,Kansas City,Leawood,3500 College Boulevard,ipo,Euronet Worldwide provides electronic payment ...,...,NaN,NaN,8642198d-0612-ceee-b600-d923394e163a,9/15/10 12:43,4/17/18 9:29,company,organization,http://venturesafrica.com/bank-of-mozambique-e...,\nThe Bank of Mozambique and US-based provider...,0
9,Clarus Corporation,company,/organization/clarus-corporation-2,USA,GA,NaN,Atlanta,NaN,ipo,A comprehensive suite of world-class financial...,...,NaN,"SQL Financials International,Black Diamond, Inc.",58bf2ec8-a615-b51f-30bf-0d028b5e426a,2/5/16 7:53,7/18/18 7:19,company,organization,NaN,NaN,0


In [40]:
for i in range(len(df['permalink'])):
    try:
        if(len(df.loc[i, 'news_article_links']) == 0):
            companyName = df['permalink'][i].lower().split('/')[-1]
            company = cb.organization(companyName)
            all_news_urls = []
            more_news = company.news

            more_news = cb.more(more_news)
            news_urls = [news.url for news in more_news]
            all_news_urls = all_news_urls + news_urls[0:1]
            if(len(all_news_urls) == 0):
                df.loc[i, 'news_article_links'] = ''
            else:
                df.loc[i, 'news_article_links'] = all_news_urls[0]
            print(str(i), df['company_name'][i])
    except:
        print(str(i), "error", df['permalink'][i])

4 error /organization/software-2000
7 error /organization/yahoo-pro
9 error /organization/clarus-corporation-2
15 error /organization/immersion-corporation
16 error /organization/f5-networks
19 error /organization/morningstar
21 error /organization/comtex-news-network
23 error /organization/omnicomm-systems
24 error /organization/sonic-foundry
25 error /organization/commvault
26 error /organization/healthgrades
27 Rediff
28 Endurance International
29 Kinaxis
30 Google
31 Quokka Sports
32 error /organization/vastera
33 The Nielsen Company
34 error /organization/salesforce
35 error /organization/dena
36 error /organization/nippon-office-systems-limited
37 error /organization/51job
38 Direct Insite
39 error /organization/qinetiq
40 error /organization/fixstars
41 error /organization/rackable-systems
42 error /organization/citrix-systems
44 error /organization/altus-pharmaceuticals
45 error /organization/altair-engineering
46 error /organization/cgg-sa
51 error /organization/bartronics
52 

308 error /organization/net1
309 error /organization/paragon-id
310 error /organization/zoom-telephonics
311 error /organization/fronteo-usa
312 error /organization/stg-group
313 HPIL Holding
314 Upland Software
315 Ackroo Canada
316 Snipp Interactive
317 FacePhi
318 error /organization/motus-gi
319 root9B Technologies
325 error /organization/ecsc-group
326 Pluralsight
331 error /organization/iteknik-holding
333 error /organization/drone-delivery-canada
335 error /organization/versarien
336 error /organization/riwi
337 error /organization/dropbox
340 error /organization/virtusa
341 error /organization/inspro
347 BTCS
348 Walker Innovation
349 Loto Interactive Ltd.
350 Zscaler
351 error /organization/boohoo-com
352 error /organization/biondvax
353 error /organization/zai-lab
354 CARE Ratings
355 error /organization/ekinops
356 error /organization/metrofile-holdings
357 Tenable Network Security
358 error /organization/bluechiip-limited
359 error /organization/pushpay
360 error /organizat

609 error /organization/counterpath
610 error /organization/crossword-cybersecurity
611 error /organization/crown-holdings
612 error /organization/cs-communication-systemes
613 error /organization/csg-limited
614 error /organization/cspi
615 error /organization/csu-cardsystem
616 error /organization/cynk-technology
617 error /organization/d4t4-solutions
618 error /organization/dmgt-plc
619 error /organization/data-exchange
620 error /organization/datametrex
621 error /organization/datatec
622 error /organization/dealnet-capital
623 error /organization/declout
624 error /organization/descartes-systems-group
625 error /organization/destiny-media-technologies
626 error /organization/deutsche-effecten-und-wechsel
627 error /organization/devoteam
628 error /organization/dgb-group
629 error /organization/diagnos
632 error /organization/digital-brand-media-marketing-group-inc
634 error /organization/digital-ga
635 error /organization/ixonos
636 error /organization/digital-lightwave
637 error 

879 error /organization/muse-technologies
880 error /organization/mvise-ag
881 error /organization/nanosonics
882 error /organization/navinfo
883 error /organization/nec
884 error /organization/ness-technologies
885 error /organization/net263
886 Netel Technology
887 Netgem
888 NetLinkz
889 error /organization/netobjects
890 error /organization/net-one-systems
891 Neurones
892 error /organization/newmark-security
893 New Venturetec
894 error /organization/nic
895 error /organization/nice-systems
896 error /organization/nicolas-miguet-et-associes-sa
897 error /organization/nid
898 error /organization/nippon-manufacturing-service
899 error /organization/nippon-development-systems
900 error /organization/nippon-telegraph-and-telephone-corporation
901 error /organization/nissha-printing
902 error /organization/nitto-denko
903 error /organization/nivs-intellimedia-technology-group
904 error /organization/norcom-informationstechnogie-ag
905 error /organization/northamber
906 Northrop Grumman

1128 error /organization/virtualsoft-systems-ltd
1129 error /organization/vodacom
1130 error /organization/vse-corp
1131 error /organization/vxl-technologies-limited
1132 error /organization/waterlink
1133 error /organization/wavestone
1134 error /organization/wayside-technology-group
1135 error /organization/weathernews
1136 error /organization/webis-holdings-plc
1138 error /organization/westminster-group-plc
1139 error /organization/whitehawk
1141 error /organization/willis-towers-watson
1142 error /organization/wilmington-2
1143 error /organization/wise-group-ab
1144 error /organization/wolters-kluwer
1145 error /organization/workday-ventures
1147 error /organization/worldwide-internet
1148 error /organization/xenia-venture-capital
1151 error /organization/xpel-technologies-corp
1152 error /organization/yokogawa
1153 Yusys Technologies
1154 ZIM
1155 Zix Corporation
1156 ZJ Innopark (Zhangjiang Hi-Tech Park)
1157 error /organization/zoomermedia


In [107]:
df['news_article_links'].fillna('', inplace=True)
df.loc[5
       , 'news_article_links']

'https://www.prnewswire.com/news-releases/liquid-web-partners-with-vmware-to-offer-a-flexible-secure-scalable-managed-private-cloud-300785668.html'

In [126]:
# df['news_article_links'].fillna('', inplace=True)
for i in range(len(df)):
    print(i)
    if(df.loc[i, 'news_article_links'] == ''):
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
        print(df.loc[i, 'permalink'])
        url = "https://www.google.com/search?q=" +df['company_name'][i].lower().split(' ')[0] + "&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiclaeihp_gAhUPPN8KHZMNDpgQ_AUIESgE&biw=1484&bih=917"
            
        print(urls[0])

        ahrefs = []

#         for j in range(10, 100, 10):
#             urls.append(urls[0]+'&start='+str(j))



        req = requests.get(url, headers=headers)

        soup = BeautifulSoup(req.content, "lxml")
        links = soup.findAll("a")

        l = []

        for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
            l.append(re.split(":(?=http)",link["href"].replace("/url?q=",""))[0])

        print(l)
#             soup = bs4.BeautifulSoup(req.text, "html.parser")

#             #you don't even have to process the div container
#             #just go strait to <a> and using indexing get "href"
#             #headlines
#             ahref  = [a["href"] for a in soup.find_all("a", class_="lLrAF")]
#             for a in ahref:
#                 ahrefs.append(a)
                

  
        if(len(ahrefs) != 0):
            df['news_article_links'][i] = l[0]

        print(str(i), df.loc[i, 'company_name'])
            

0
1
2
3
4
/organization/software-2000
https://www.google.com/search?q=citrix&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiclaeihp_gAhUPPN8KHZMNDpgQ_AUIESgE&biw=1484&bih=917
[]
4 Infinium Software
5
6
7
/organization/yahoo-pro
https://www.google.com/search?q=citrix&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiclaeihp_gAhUPPN8KHZMNDpgQ_AUIESgE&biw=1484&bih=917
[]
7 Yahoo Pro
8
9
/organization/clarus-corporation-2
https://www.google.com/search?q=citrix&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiclaeihp_gAhUPPN8KHZMNDpgQ_AUIESgE&biw=1484&bih=917
[]
9 Clarus Corporation
10
/organization/verisign-2
https://www.google.com/search?q=citrix&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiclaeihp_gAhUPPN8KHZMNDpgQ_AUIESgE&biw=1484&bih=917
[]
10 VeriSign
11
12
13
14
15
/organization/immersion-corporation
https://www.google.com/search?q=citrix&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiclaeihp_gAhUPPN8KHZMNDpgQ_AUIESgE&biw=1484&bih=917
[]
15 Immersion Corporation
16
/organization/f5-networks
https://www.google.com/search?q=citrix&source

KeyboardInterrupt: 

In [73]:
df

,company_name,roles,permalink,country_code,state_code,region,city,address,status,short_description,...,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type,news_article_links,article,sentiment_polarity
0,Intel,"company,investor",/organization/intel,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,ipo,"Intel designs, manufactures, and sells integra...",...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2/6/08 6:28,12/11/18 10:52,company,organization,,,0
1,Oracle,"company,investor",/organization/oracle,USA,CA,SF Bay Area,Redwood Shores,500 Oracle Parkway,ipo,Oracle is an integrated cloud applications and...,...,https://twitter.com/Oracle,NaN,bef5bd4b-72c6-7877-d7ab-8bbe43f7bda7,2/28/08 0:16,1/4/19 9:07,company,organization,,,0
2,Kainos,"company,investor",/organization/kainos-software,GBR,NaN,Belfast,Belfast,4-6 Upper Crescent,ipo,Kainos provide digital technology solutions th...,...,http://twitter.com/KainosSoftware,Kainos Group Plc,a307ed6d-2605-328a-e50a-d83f82871ac7,8/15/13 14:40,4/17/18 9:29,company,organization,,,0
3,Progress,company,/organization/progress-software,USA,MA,Boston,Bedford,14 Oak Park Drive,ipo,Progress is a global software company that sim...,...,https://twitter.com/ProgressSW,Progress Software,ac1b64f5-4012-c3f9-59c0-1f48f1df3f5c,4/10/08 7:39,10/2/18 23:54,company,organization,,,0
4,Infinium Software,company,/organization/software-2000,USA,MA,NaN,Hyannis Port,NaN,ipo,Infinium Software provides enterprise business...,...,NaN,Software 2000,01915de2-7ae6-9787-e397-855a71b1fafe,9/13/13 12:45,2/12/18 23:35,company,organization,,,0
5,NetApp,"company,investor",/organization/netapp,USA,CA,SF Bay Area,Sunnyvale,495 East Java Drive,ipo,NetApp is a hybrid cloud data services company .,...,http://twitter.com/netapp,Net App,bbd3ae58-58e2-d81c-42bf-97d4a07ac0d1,1/4/08 15:41,11/27/18 21:57,company,organization,,,0
6,Veon,company,/organization/vimpelcom,NLD,NaN,Amsterdam,Amsterdam,"SOM 2 Bld., Floor 2, Claude Debussylaan 15",ipo,Veon is an international communications and te...,...,https://twitter.com/veondigital,"FGI Wireless,VimpelCom",3565d953-9352-8281-302d-721fd05a0f6a,5/6/11 19:15,10/27/18 0:25,company,organization,,,0
7,Yahoo Pro,company,/organization/yahoo-pro,USA,CA,NaN,Sunnyvale,NaN,ipo,Yahoo is a technology company that is known fo...,...,https://twitter.com/Yahoo,NaN,c30b2e7d-a46e-4c22-9533-c27efff502ae,11/12/18 21:16,11/13/18 18:42,company,organization,,,0
8,Euronet Worldwide,"company,investor",/organization/euronet-worldwide,USA,KS,Kansas City,Leawood,3500 College Boulevard,ipo,Euronet Worldwide provides electronic payment ...,...,NaN,NaN,8642198d-0612-ceee-b600-d923394e163a,9/15/10 12:43,4/17/18 9:29,company,organization,,,0
9,Clarus Corporation,company,/organization/clarus-corporation-2,USA,GA,NaN,Atlanta,NaN,ipo,A comprehensive suite of world-class financial...,...,NaN,"SQL Financials International,Black Diamond, Inc.",58bf2ec8-a615-b51f-30bf-0d028b5e426a,2/5/16 7:53,7/18/18 7:19,company,organization,,,0


In [70]:
df[0:50]

,company_name,roles,permalink,country_code,state_code,region,city,address,status,short_description,...,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type,news_article_links,article,sentiment_polarity
0,Intel,"company,investor",/organization/intel,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,ipo,"Intel designs, manufactures, and sells integra...",...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2/6/08 6:28,12/11/18 10:52,company,organization,http://www.cbj.ca/development-timelines-and-tr...,"\nCBJ Newsmakers\nVANCOUVER, British Columbia,...",0
1,Oracle,"company,investor",/organization/oracle,USA,CA,SF Bay Area,Redwood Shores,500 Oracle Parkway,ipo,Oracle is an integrated cloud applications and...,...,https://twitter.com/Oracle,NaN,bef5bd4b-72c6-7877-d7ab-8bbe43f7bda7,2/28/08 0:16,1/4/19 9:07,company,organization,https://techcrunch.com/2018/11/29/the-white-ho...,\n\nThe White House will meet with tech execs ...,0
2,Kainos,"company,investor",/organization/kainos-software,GBR,NaN,Belfast,Belfast,4-6 Upper Crescent,ipo,Kainos provide digital technology solutions th...,...,http://twitter.com/KainosSoftware,Kainos Group Plc,a307ed6d-2605-328a-e50a-d83f82871ac7,8/15/13 14:40,4/17/18 9:29,company,organization,https://www.pehub.com/2019/01/pe-hub-wire-high...,\n\n \n\n\t\t\t\t\t\t\t\tPhoto of Luisa Beltra...,0
3,Progress,company,/organization/progress-software,USA,MA,Boston,Bedford,14 Oak Park Drive,ipo,Progress is a global software company that sim...,...,https://twitter.com/ProgressSW,Progress Software,ac1b64f5-4012-c3f9-59c0-1f48f1df3f5c,4/10/08 7:39,10/2/18 23:54,company,organization,https://www.huffingtonpost.com/entry/max-rose-...,"\nDemocrat Max Rose, a 31-year-old former Army...",0
4,Infinium Software,company,/organization/software-2000,USA,MA,NaN,Hyannis Port,NaN,ipo,Infinium Software provides enterprise business...,...,NaN,Software 2000,01915de2-7ae6-9787-e397-855a71b1fafe,9/13/13 12:45,2/12/18 23:35,company,organization,NaN,NaN,0
5,NetApp,"company,investor",/organization/netapp,USA,CA,SF Bay Area,Sunnyvale,495 East Java Drive,ipo,NetApp is a hybrid cloud data services company .,...,http://twitter.com/netapp,Net App,bbd3ae58-58e2-d81c-42bf-97d4a07ac0d1,1/4/08 15:41,11/27/18 21:57,company,organization,https://www.prnewswire.com/news-releases/liqui...,"\nLANSING, Mich., Jan. 29, 2019 /PRNewswire/ -...",0
6,Veon,company,/organization/vimpelcom,NLD,NaN,Amsterdam,Amsterdam,"SOM 2 Bld., Floor 2, Claude Debussylaan 15",ipo,Veon is an international communications and te...,...,https://twitter.com/veondigital,"FGI Wireless,VimpelCom",3565d953-9352-8281-302d-721fd05a0f6a,5/6/11 19:15,10/27/18 0:25,company,organization,https://www.prnewswire.com/news-releases/veon-...,"\nAMSTERDAM, Jan. 23, 2019 /PRNewswire/ -- VEO...",0
7,Yahoo Pro,company,/organization/yahoo-pro,USA,CA,NaN,Sunnyvale,NaN,ipo,Yahoo is a technology company that is known fo...,...,https://twitter.com/Yahoo,NaN,c30b2e7d-a46e-4c22-9533-c27efff502ae,11/12/18 21:16,11/13/18 18:42,company,organization,NaN,NaN,0
8,Euronet Worldwide,"company,investor",/organization/euronet-worldwide,USA,KS,Kansas City,Leawood,3500 College Boulevard,ipo,Euronet Worldwide provides electronic payment ...,...,NaN,NaN,8642198d-0612-ceee-b600-d923394e163a,9/15/10 12:43,4/17/18 9:29,company,organization,http://venturesafrica.com/bank-of-mozambique-e...,\nThe Bank of Mozambique and US-based provider...,0
9,Clarus Corporation,company,/organization/clarus-corporation-2,USA,GA,NaN,Atlanta,NaN,ipo,A comprehensive suite of world-class financial...,...,NaN,"SQL Financials International,Black Diamond, Inc.",58bf2ec8-a615-b51f-30bf-0d028b5e426a,2/5/16 7:53,7/18/18 7:19,company,organization,NaN,NaN,0


In [50]:
df['news_article_links'][90]

''

In [22]:
df.to_csv('csv_export/temp1.5.csv', index=False)

In [14]:
for i in range(len(df['news_article_links'])):
    try:
        url = df['news_article_links'][i]
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
        opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
        html =  opener.open(url).read().decode('utf-8')
        readable_article = Document(html).summary()
        soup = BeautifulSoup(readable_article, "lxml")
        text = soup.get_text()
        df.loc[i, 'article'] = text
        print(str(i), df['company_name'][i])
    except:
        print(str(i), "error", df['permalink'][i])

0 Intel
1 Oracle
2 Kainos
3 Progress
4 error /organization/software-2000
5 NetApp
6 Veon
7 error /organization/yahoo-pro
8 Euronet Worldwide
9 error /organization/clarus-corporation-2
10 VeriSign
11 Bechtle
12 AsiaInfo
13 Extreme Networks
14 Cambridge Technology Enterprises
15 error /organization/immersion-corporation
16 error /organization/f5-networks
17 NetScout Systems
18 eGain Corporation
19 error /organization/morningstar
20 Nuance Communications
21 error /organization/comtex-news-network
22 error /organization/network
23 error /organization/omnicomm-systems
24 error /organization/sonic-foundry
25 error /organization/commvault
26 error /organization/healthgrades
27 error /organization/rediff
28 Endurance International
29 error /organization/kinaxis
30 Google
31 Quokka Sports
32 error /organization/vastera
33 The Nielsen Company
34 error /organization/salesforce
35 error /organization/dena
36 error /organization/nippon-office-systems-limited
37 error /organization/51job
38 Direct I

264 Tejas Networks India
265 error /organization/intellinetics
266 error /organization/just-dial
267 Engineering Ingegneria Informatica SpA
268 error /organization/intercloud-systems
269 error /organization/smtp
270 error /organization/atm
271 error /organization/securecare-technologies-inc
272 error /organization/tikforce
273 error /organization/yowie
274 Emerald Medical Applications
275 Axway
276 Alibaba Group
277 error /organization/pinnacle-technologies
278 eir
279 error /organization/gokuldas-exports-ltd
280 error /organization/vmob
281 Digiliti Money
282 Lantronix
283 error /organization/nanotech-security
284 error /organization/lattice-incorporated
285 error /organization/otherlevels
286 error /organization/ameri-100
287 Bigblu Broadband
288 error /organization/teladoc
289 error /organization/adesto-technologies
290 error /organization/great-basin-corporation
291 error /organization/l-t-infotech
292 error /organization/bidenergy
293 error /organization/neusoft
294 error /organiz

562 B-Scada, Inc.
563 error /organization/showa-ota-co
564 CAC Corporation
565 error /organization/california-software
566 error /organization/canadian-zeolite
567 error /organization/candente-copper
568 error /organization/cannabix-technologies
569 Capgemini Group
570 error /organization/capinfo-company
571 error /organization/carl-zeiss-meditec
572 error /organization/cass-information-systems
573 error /organization/catalyst-media-group-170e
574 error /organization/catalyst-health-solutions-inc
575 error /organization/milestone-group-plc
576 error /organization/catenon-international
577 Cegedim
578 error /organization/celsys
579 error /organization/cerinnov
580 CGI
581 CHC Helicopter
582 error /organization/cheops-technology
583 China ITS Holdings Co Ltd
584 error /organization/chinalink-professional-service
585 error /organization/china-telecom-corporation
586 error /organization/china-transinfo-technology
587 error /organization/cibox
588 error /organization/cistera
589 error /orga

804 ITT Corporation
805 error /organization/ivalis-sa
806 error /organization/ivow
807 error /organization/ixup
808 error /organization/jacobs
809 error /organization/jardine-matheson-group
810 error /organization/jasco-group
811 error /organization/jbcc-holdings
812 error /organization/jig-saw
813 error /organization/john-laing-group-plc
814 error /organization/johnson-johnson
815 error /organization/johnson-matthey
816 error /organization/johnston-press
817 error /organization/calanit
818 error /organization/keyence
819 error /organization/kingtone-wireless
820 error /organization/knosys
821 error /organization/kozo-keikaku-engineering
822 error /organization/kpn
823 KRM22
824 Krones
825 error /organization/kronologi
826 error /organization/kuraray
827 L3 Harris Technologies
828 Leidos holdings
829 error /organization/leo-group
830 error /organization/lg
831 error /organization/liandi-clean-technology
832 error /organization/lict-corporation
833 error /organization/linedata
834 error

1048 Spok Holdings
1049 error /organization/sqli-group
1050 error /organization/startia-inc
1051 error /organization/statpro
1052 error /organization/stilo-international
1053 error /organization/stone-pagamentos-sa
1054 error /organization/strategic-resources-inc-sri
1055 error /organization/sutimco
1056 error /organization/suzhou-jinfu-technology-co
1057 error /organization/svoa
1058 error /organization/swisscom
1059 error /organization/switch-6
1060 error /organization/sycamore-networks-2
1061 error /organization/sykes-enterprises
1062 error /organization/sylogist-ltd
1063 Synectics
1064 error /organization/synnex-2
1065 SYNNEX
1066 Synopsys
1067 error /organization/sysgroup-plc
1068 error /organization/sysorex-8852
1069 error /organization/tapuz
1070 Tata Consultancy Services
1071 error /organization/tbank
1072 error /organization/tech-data
1073 error /organization/techfirm-inc
1074 error /organization/technical-communication-corporation
1075 error /organization/techniche
1076 error

In [25]:
df['article'].fillna('', inplace=True)
df = df.replace({'\xa0': ' '}, regex=True) # clean these empty null characters.
for i in range(len(df)):
    if len(df['article'][i]) < 10:
        df = df.drop(i)
df = df.reset_index(drop=True)
df

,company_name,roles,permalink,country_code,state_code,region,city,address,status,short_description,...,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type,news_article_links,article,sentiment_polarity
0,Intel,"company,investor",/organization/intel,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,ipo,"Intel designs, manufactures, and sells integra...",...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2/6/08 6:28,12/11/18 10:52,company,organization,http://www.cbj.ca/development-timelines-and-tr...,"\nCBJ Newsmakers\nVANCOUVER, British Columbia,...",0
1,Oracle,"company,investor",/organization/oracle,USA,CA,SF Bay Area,Redwood Shores,500 Oracle Parkway,ipo,Oracle is an integrated cloud applications and...,...,https://twitter.com/Oracle,NaN,bef5bd4b-72c6-7877-d7ab-8bbe43f7bda7,2/28/08 0:16,1/4/19 9:07,company,organization,https://techcrunch.com/2018/11/29/the-white-ho...,\n\nThe White House will meet with tech execs ...,0
2,Kainos,"company,investor",/organization/kainos-software,GBR,NaN,Belfast,Belfast,4-6 Upper Crescent,ipo,Kainos provide digital technology solutions th...,...,http://twitter.com/KainosSoftware,Kainos Group Plc,a307ed6d-2605-328a-e50a-d83f82871ac7,8/15/13 14:40,4/17/18 9:29,company,organization,https://www.pehub.com/2019/01/pe-hub-wire-high...,\n\n \n\n\t\t\t\t\t\t\t\tPhoto of Luisa Beltra...,0
3,Progress,company,/organization/progress-software,USA,MA,Boston,Bedford,14 Oak Park Drive,ipo,Progress is a global software company that sim...,...,https://twitter.com/ProgressSW,Progress Software,ac1b64f5-4012-c3f9-59c0-1f48f1df3f5c,4/10/08 7:39,10/2/18 23:54,company,organization,https://www.huffingtonpost.com/entry/max-rose-...,"\nDemocrat Max Rose, a 31-year-old former Army...",0
4,NetApp,"company,investor",/organization/netapp,USA,CA,SF Bay Area,Sunnyvale,495 East Java Drive,ipo,NetApp is a hybrid cloud data services company .,...,http://twitter.com/netapp,Net App,bbd3ae58-58e2-d81c-42bf-97d4a07ac0d1,1/4/08 15:41,11/27/18 21:57,company,organization,https://www.prnewswire.com/news-releases/liqui...,"\nLANSING, Mich., Jan. 29, 2019 /PRNewswire/ -...",0
5,Veon,company,/organization/vimpelcom,NLD,NaN,Amsterdam,Amsterdam,"SOM 2 Bld., Floor 2, Claude Debussylaan 15",ipo,Veon is an international communications and te...,...,https://twitter.com/veondigital,"FGI Wireless,VimpelCom",3565d953-9352-8281-302d-721fd05a0f6a,5/6/11 19:15,10/27/18 0:25,company,organization,https://www.prnewswire.com/news-releases/veon-...,"\nAMSTERDAM, Jan. 23, 2019 /PRNewswire/ -- VEO...",0
6,Euronet Worldwide,"company,investor",/organization/euronet-worldwide,USA,KS,Kansas City,Leawood,3500 College Boulevard,ipo,Euronet Worldwide provides electronic payment ...,...,NaN,NaN,8642198d-0612-ceee-b600-d923394e163a,9/15/10 12:43,4/17/18 9:29,company,organization,http://venturesafrica.com/bank-of-mozambique-e...,\nThe Bank of Mozambique and US-based provider...,0
7,VeriSign,"company,investor",/organization/verisign-2,USA,VA,VA - Other,Bluemont,12061 Bluemont Way,ipo,Verisign operates the infrastructure for the ....,...,http://twitter.com/VERISIGN,NaN,57f25563-7693-bb4f-3cb4-bedecf93b13f,5/26/14 10:42,10/27/18 0:22,company,organization,,\nEvery day we send out a free e-mail with the...,0
8,Bechtle,company,/organization/bechtle,DEU,NaN,DEU - Other,Neckarsulm,Bechtle Platz 1,ipo,The Bechtle IT system houses provide their cli...,...,https://twitter.com/bechtle_ag,NaN,cb7016e8-3517-0259-7c9b-0e092b78d743,5/22/17 6:17,9/15/18 8:57,company,organization,https://uk.reuters.com/article/brief-bechtle-b...,Jan 8 (Reuters) - Bechtle AG: * BUYS ACOMMIT A...,0
9,AsiaInfo,company,/organization/asiainfo,CHN,NaN,Beijing,Beijing,"East Area, No. 10 Xibeiwang East Road, Haidian...",ipo,AsiaInfo is an innovative provider of software...,...,http://twitter.com/AsiainfoLinkage,NaN,43f794ca-2a16-9550-a8bf-a0f6d8b96222,6/26/13 14:07,10/26/18 21:34,company,organization,https://www.avcj.com/avcj/news/3011379/chinas-...,"\n\n\n\n\nAsia

In [129]:
articles = [str(s) for s in df['article']] # create a corpus of all articles
articles[0]

'\nCBJ Newsmakers\nVANCOUVER, British Columbia, Feb.  01, 2019  (GLOBE NEWSWIRE) — MLA Canada has released its annual Market Intel real estate report, highlighting last year as a pivotal point in the Greater Vancouver real estate market. The 2019 report by MLA Canada’s Analytics and Advisory group analyzes the outgoing year, trends, current market conditions and provides a forecast for the year ahead.\nThe report includes major economic trends locally, interprovincially and internationally. Surprising data from the report includes a breakdown of the true cost of a home, the rapid rise of construction cost, as well an inside look at City approval timelines and purchaser and population insights.\n“BC’s fundamentals remain stronger than elsewhere in the country. This means pressures on our housing market will continue, particularly with population increasing and housing starts declining.” says Cameron McNeill, Executive Director and Partner at MLA Canada. “We must be progressive in our so

In [130]:
# load nltk's English stopwords
stopwords = nltk.corpus.stopwords.words('english')
# load nltk's SnowballStemmer
stemmer = SnowballStemmer("english")

# Define a tokenizer and stemmer that returns the set of stems of the text
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if len(token) <= 1: # remove one character words, "a" apostraphe s.
            continue
        if token.isalpha():
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if len(token) <= 1: # remove one character words, "a" apostraphe s.
            continue
        if token.isalpha():
            filtered_tokens.append(token)
    return filtered_tokens

In [131]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in articles:
    allwords_stemmed = tokenize_and_stem(i) #for each article in "articles", tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
vocab_df = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed) # map stem to first actual word for simplicity
print('Shape of vocab_df (rows, columns):', vocab_df.shape, '\n')
vocab_df.head()

Shape of vocab_df (rows, columns): (156158, 1) 



,words
cbj,cbj
newsmak,newsmakers
vancouv,vancouver
british,british
columbia,columbia


In [132]:
stopwords = set(w.lower() for w in nltk.corpus.stopwords.words('english'))

# From: http://www.nltk.org/howto/sentiment.html
def sentiment_polarity(s):
    paragraphs = re.split('\n' , s)
    paragraphs = [p for p in paragraphs if p != '']
    sid = SentimentIntensityAnalyzer()
    neu = []
    compound = []
    for sentence in paragraphs:
        ss = sid.polarity_scores(sentence)
        neu.append(ss['neu'])
    return np.var(neu)

for i in range(len(df['sentiment_polarity'])):
    try:
        df.loc[i, 'sentiment_polarity'] = sentiment_polarity(df['article'][i])
        print(i, ':', df['news_article_links'][i], ":", sentiment_polarity(df['article'][i]))
    except:
        pass # keeps empty articles 0.0

0 : http://www.cbj.ca/development-timelines-and-true-construction-costs-at-the-forefront-of-real-estate-intel-report-2/ : 0.006673694444444443
1 : https://techcrunch.com/2018/11/29/the-white-house-will-meet-with-tech-execs-to-talk-transformational-ideas/ : 0.0053806666666666655
2 : https://www.pehub.com/2019/01/pe-hub-wire-highlights-1-29-19/ : 0.09154285937500001
3 : https://www.huffingtonpost.com/entry/max-rose-staten-island-congress_us_5b9e4943e4b046313fbbf578 : 0.007561319615912209
5 : https://www.prnewswire.com/news-releases/liquid-web-partners-with-vmware-to-offer-a-flexible-secure-scalable-managed-private-cloud-300785668.html : 0.016065466666666663
6 : https://www.prnewswire.com/news-releases/veon-reaffirms-commitment-to-addressing-strategic-relationship-with-gth-300782761.html : 0.003204109375000001
8 : http://venturesafrica.com/bank-of-mozambique-euronet-sign-deal-to-set-up-interbank-electronic-payment-system/ : 0.004636210000000002
10 :  : 0.12037688888888891
11 : https://uk.

217 : https://www.prweb.com/releases/2018/06/prweb15563000.htm : 0.01736561082206035
224 : https://www.prnewswire.com/news-releases/nobilis-health-corp-names-phil-ayres-chief-marketing-officer-300653134.html : 0.00586659171597633
225 : http://www.prweb.com/releases/2018/05/prweb15480062.htm : 0.0135363125
226 : http://www.techcrunch.com/2008/12/08/outlook-replacement-unison-brings-ad-supported-email-to-small-businesses/ : 0.0054901388888888856
227 : http://www.thisismoney.co.uk/money/markets/article-5963713/DAILY-BRIEFING-Uber-faces-investigation-gender-discrimination-complaints.html : 0.08363823140495869
232 : https://www.androidheadlines.com/2019/01/ooma-smart-security-camera.html : 0.11413359999999999
237 : http://www.businesswire.com/news/home/20150618005162/en/Actual-Experience-Announces-Key-Corporate-Milestones : 0.12054973287197235
241 : http://cicerostaging.wpengine.com/nexidia-and-cicero-announce-partnership : 0.013676952520087658
242 : https://www.telecompaper.com/news/daisy-

395 : https://www.akbizmag.com/monitor/exxonmobil-reg-partner-with-clariant/ : 0.0073135555555555514
401 : https://economictimes.indiatimes.com/tech/ites/wipro-plans-to-double-campus-hiring-this-year-as-company-sees-growth/articleshow/67717414.cms : 0.06505039053254438
403 : https://www.livemint.com/Companies/H2UAdCnmjHTQ5ZYSrqXmcN/How-Kumar-Mangalam-Birla-salvaged-the-Hindalco-Aleris-deal.html : 0.0029639669421487615
404 : https://www.arnnet.com.au/mediareleases/33596/anaplan-named-a-leader-in-gartner-magic-quadrant/ : 0.06133263265306121
407 : https://www.prnewswire.com/news-releases/maxar-technologies-ssl-terminates-its-participation-in-darpas-robotic-servicing-of-geosynchronous-satellites-program-300786378.html : 0.004168243055555558
409 : https://www.prnewswire.com/news-releases/iovation-report-finds-online-gambling-mobile-use-and-fraud-increase-in-2018-300787451.html : 0.029524284023668634
412 : http://www.cbj.ca/quorum-announces-acquisition-of-oasis-auto-complete-systems-limited

661 : https://www.prnewswire.com/news-releases/entree-resources-announces-fiscal-year-2017-results-and-reviews-corporate-highlights-676426513.html : 0.027479769586471126
674 : http://www.prnewswire.com/news-releases/enterprise-content-management-market-2017-2023-with-adobe-everteam-fabasoft-hyland-software--ibm-laserfiche-microsoft-open-text-oracle-and-xerox-dominating-300506049.html : 0.001620013605442177
675 : http://en.acnnewswire.com/press-release/english/40498/far-east-horizon---a-new-star-in-china's-healthcare-market : 0.22089125440000001
682 : http://amppob.com/demo-day-for-fintech-accelerator-startups/ : 0.015880354166666666
692 : https://www.prnewswire.com/news-releases/fujitsu-computer-products-of-america-inc-announces-the-first-scanner-subscription-service-in-the-industry-300787813.html : 0.013664639999999999
693 : https://www.businesswire.com/news/home/20180711005220/en/Leading-Fintech-Payment-Group-Adopts-Fusionex-Big : 0.11746228501077255
696 : http://www.galantas.com/new

992 : https://businesswireindia.com/news/fulldetails/rockwell-automation-acquires-emulate3d-leading-software-developer-simulating-emulating-industrial-automation-systems/61661 : 0.1574166304
998 : https://www.prnewswire.com/news-releases/2019-advanced-engine-technologies-for-meeting-co2-and-fuel-economy-targets---forecast-to-2033-300776310.html : 0.026453451595457003
999 : https://www.hollywoodreporter.com/news/men-are-embracing-brooches-red-carpet-1180468 : 0.009967416666666664
1011 : https://talkbusiness.net/2019/01/serco-leased-property-in-rogers-sells-for-6-5-million/ : 0.003558743055555556
1018 : https://www.nytimes.com/2018/03/08/business/japan-women-construction.html : 0.0
1023 : https://www.prnewswire.com/news-releases/tier-1-internet-and-content-provider-chooses-silicoms-switch-on-a-nic-solution-for-its-sd-wan-related-distribution-hub-infrastructure-300671371.html : 0.013887209183673468
1024 : http://www.marketwired.com/press-release/silver-bull-announces-closing-initial-tranc

In [133]:
import csv
df.to_csv('csv_export/temp2.5.csv', index=False)
# TODO: commas

In [41]:
blah = pd.read_csv('csv_export/temp2.5.csv', sep="|")
blah

,company_name,roles,permalink,country_code,state_code,region,city,address,status,short_description,...,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type,news_article_links,article,sentiment_polarity
0,Intel,"company,investor",/organization/intel,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,ipo,"Intel designs, manufactures, and sells integra...",...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2/6/08 6:28,12/11/18 10:52,company,organization,http://www.cbj.ca/development-timelines-and-tr...,"\nCBJ Newsmakers\nVANCOUVER, British Columbia,...",0.006674
1,Oracle,"company,investor",/organization/oracle,USA,CA,SF Bay Area,Redwood Shores,500 Oracle Parkway,ipo,Oracle is an integrated cloud applications and...,...,https://twitter.com/Oracle,NaN,bef5bd4b-72c6-7877-d7ab-8bbe43f7bda7,2/28/08 0:16,1/4/19 9:07,company,organization,https://techcrunch.com/2018/11/29/the-white-ho...,\n\nThe White House will meet with tech execs ...,0.005381
2,Kainos,"company,investor",/organization/kainos-software,GBR,NaN,Belfast,Belfast,4-6 Upper Crescent,ipo,Kainos provide digital technology solutions th...,...,http://twitter.com/KainosSoftware,Kainos Group Plc,a307ed6d-2605-328a-e50a-d83f82871ac7,8/15/13 14:40,4/17/18 9:29,company,organization,https://www.pehub.com/2019/01/pe-hub-wire-high...,\n\n \n\n\t\t\t\t\t\t\t\tPhoto of Luisa Beltra...,0.091543
3,Progress,company,/organization/progress-software,USA,MA,Boston,Bedford,14 Oak Park Drive,ipo,Progress is a global software company that sim...,...,https://twitter.com/ProgressSW,Progress Software,ac1b64f5-4012-c3f9-59c0-1f48f1df3f5c,4/10/08 7:39,10/2/18 23:54,company,organization,https://www.huffingtonpost.com/entry/max-rose-...,"\nDemocrat Max Rose, a 31-year-old former Army...",0.007561
4,NetApp,"company,investor",/organization/netapp,USA,CA,SF Bay Area,Sunnyvale,495 East Java Drive,ipo,NetApp is a hybrid cloud data services company .,...,http://twitter.com/netapp,Net App,bbd3ae58-58e2-d81c-42bf-97d4a07ac0d1,1/4/08 15:41,11/27/18 21:57,company,organization,https://www.prnewswire.com/news-releases/liqui...,"\nLANSING, Mich., Jan. 29, 2019 /PRNewswire/ -...",0.016065
5,Veon,company,/organization/vimpelcom,NLD,NaN,Amsterdam,Amsterdam,"SOM 2 Bld., Floor 2, Claude Debussylaan 15",ipo,Veon is an international communications and te...,...,https://twitter.com/veondigital,"FGI Wireless,VimpelCom",3565d953-9352-8281-302d-721fd05a0f6a,5/6/11 19:15,10/27/18 0:25,company,organization,https://www.prnewswire.com/news-releases/veon-...,"\nAMSTERDAM, Jan. 23, 2019 /PRNewswire/ -- VEO...",0.003204
6,Euronet Worldwide,"company,investor",/organization/euronet-worldwide,USA,KS,Kansas City,Leawood,3500 College Boulevard,ipo,Euronet Worldwide provides electronic payment ...,...,NaN,NaN,8642198d-0612-ceee-b600-d923394e163a,9/15/10 12:43,4/17/18 9:29,company,organization,http://venturesafrica.com/bank-of-mozambique-e...,\nThe Bank of Mozambique and US-based provider...,0.004636
7,VeriSign,"company,investor",/organization/verisign-2,USA,VA,VA - Other,Bluemont,12061 Bluemont Way,ipo,Verisign operates the infrastructure for the ....,...,http://twitter.com/VERISIGN,NaN,57f25563-7693-bb4f-3cb4-bedecf93b13f,5/26/14 10:42,10/27/18 0:22,company,organization,NaN,\nEvery day we send out a free e-mail with the...,0.120377
8,Bechtle,company,/organization/bechtle,DEU,NaN,DEU - Other,Neckarsulm,Bechtle Platz 1,ipo,The Bechtle IT system houses provide their cli...,...,https://twitter.com/bechtle_ag,NaN,cb7016e8-3517-0259-7c9b-0e092b78d743,5/22/17 6:17,9/15/18 8:57,company,organization,https://uk.reuters.com/article/brief-bechtle-b...,Jan 8 (Reuters) - Bechtle AG: * BUYS ACOMMIT A...,0.000000
9,AsiaInfo,company,/organization/asiainfo,CHN,NaN,Beijing,Beijing,"East Area, No. 10 Xibeiwang East Road, Haidian...",ipo,AsiaInfo is an innovative provider of software...,...,http://twitter.com/AsiainfoLinkage,NaN,43f794ca-2a16-9550-a8bf-a0f6d8b96222,6/26/13 14:07,10/26/18 21:34,company,organization

In [25]:
# Define vectorizer parameters using Gridsearch (later in this notebook)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=0.01, max_df = 0.8, use_idf=True, ngram_range=(1,1),
                                   tokenizer=tokenize_and_stem)

tfidf_matrix = tfidf_vectorizer.fit_transform(articles) #fit the vectorizer to descriptions
terms = tfidf_vectorizer.get_feature_names() # terms is just a list of the features used in the tf-idf matrix.
print(tfidf_matrix.shape)
terms

/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))


(166, 3391)


['abdul',
 'abhishek',
 'abil',
 'abl',
 'aboard',
 'abov',
 'absolut',
 'absorb',
 'absorpt',
 'abund',
 'abus',
 'academ',
 'academi',
 'academia',
 'acceler',
 'accept',
 'access',
 'accident',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accumul',
 'accur',
 'accuraci',
 'ace',
 'achiev',
 'acknowledg',
 'acquir',
 'acquisit',
 'act',
 'action',
 'activ',
 'actor',
 'actual',
 'acut',
 'ad',
 'adapt',
 'add',
 'addict',
 'addit',
 'address',
 'adher',
 'adjac',
 'adjust',
 'administr',
 'admiss',
 'admit',
 'adob',
 'adopt',
 'adrian',
 'adult',
 'advanc',
 'advantag',
 'advertis',
 'advic',
 'advis',
 'advisor',
 'advisori',
 'advocaci',
 'aerospac',
 'affair',
 'affect',
 'affili',
 'afford',
 'aforement',
 'africa',
 'afternoon',
 'ag',
 'agarw',
 'age',
 'agenc',
 'agenda',
 'agent',
 'aggreg',
 'agil',
 'agili',
 'ago',
 'agre',
 'agreement',
 'agricultur',
 'agronom',
 'ahead',
 'ai',
 'aid',
 'aifloo',
 'aim',
 'air',
 'airbud',
 'airbus',
 'aircraft',


In [26]:
from sklearn.cluster import KMeans
num_clusters = 5 # played around with this number, and decided on 6 clusters / 6 main categories of the articles
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [30]:
from sklearn.externals import joblib
# joblib.dump(km, 'csv_export/km.pkl') 
km = joblib.load('csv_export/km.pkl') 

In [32]:
urls = [str(t) for t in df['news_article_links']]
categories = [c for c in df['category_list']]# just to compare the cluster and the actual category
news_articles = { 'url': urls, 'article': articles, 'cluster': clusters, 'categories': categories}
cluster_df = pd.DataFrame(news_articles, index = [clusters] , columns = ['url', 'article', 'cluster', 'categories'])
print(cluster_df['cluster'].value_counts()) #number of films per cluster (clusters from 0 to 5)
cluster_df.head()

2    55
4    41
0    30
3    21
1    19
Name: cluster, dtype: int64


,url,article,cluster,categories
0,http://www.cbj.ca/development-timelines-and-tr...,"\nCBJ Newsmakers\nVANCOUVER, British Columbia,...",0,"Artificial Intelligence,Consumer Electronics,E..."
4,https://www.csmonitor.com/The-Culture/Arts/201...,\nTidy spaceReaders whose hearts were captured...,4,"Ad Network,Artificial Intelligence,Blogging Pl..."
0,https://uk.reuters.com/article/idUKL8N1ZA0KI,Jan 10 (Reuters) - BOSSARD HOLDING AG: * FY GR...,0,"Analytics,Artificial Intelligence,Cloud Comput..."
0,http://www.techweekeurope.co.uk/workspace/micr...,Power over Ethernet (PoE) midspans from semic...,0,"Industrial Automation,Intelligent Systems,VoIP"
4,http://www.businesswire.com/news/home/20151013...,\nLONDON & TOKYO--(BUSINESS WIRE)--A new Londo...,4,"Data Visualization,Enterprise Software,Natural..."


In [33]:
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster", i, "top words:", end='')
    for ind in order_centroids[i, :10]: # 10 words per cluster
#         print(terms[ind].split(' '))
#         print(vocab_df.ix[terms[ind].split(' ')].values.tolist())
        print(vocab_df.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=', ')
    print() #add whitespace
    print() #add whitespace
    
    # USE STEM
    
#     print("Cluster", i, "titles:", end='')
#     for title in cluster_df.ix[i]['title'].values.tolist():
#         print(title, end='')
#     print() #add whitespace
#     print() #add whitespace

Cluster 0 top words:b'startups', b'company', b'technology', b'year', b'million', b'employee', b'engineering', b'developers', b'ai', b'market', 

Cluster 1 top words:b'infrastructure', b'water', b'log', b'dying', b'representatives', b'cannabis', b'using', b'docker', b'time', b'car', 

Cluster 2 top words:b'data', b'company', b'ai', b'secured', b'analytics', b'customers', b'technology', b'platforms', b'intelligence', b'using', 

Cluster 3 top words:b'iot', b'product', b'customers', b'platforms', b'users', b'company', b'app', b'smart', b'developers', b'startups', 

Cluster 4 top words:b'email', b'likely', b'using', b'london', b'virtual', b'reserved', b'company', b'right', b'says', b'team', 



/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [ ]:
df['category_list'].head()

In [ ]:
df.shape